# Deviations from Normality

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import edhec_risk_kit as erk

In [3]:
hfi = erk.get_hfi_returns()
hfi.head()

,Convertible Arbitrage,CTA Global,Distressed Securities,Emerging Markets,Equity Market Neutral,Event Driven,Fixed Income Arbitrage,Global Macro,Long/Short Equity,Merger Arbitrage,Relative Value,Funds Of Funds
date,,,,,,,,,,,,
1997-01,0.0119,0.0393,0.0178,0.0791,0.0189,0.0213,0.0191,0.0573,0.0281,0.0150,0.0180,0.0317
1997-02,0.0123,0.0298,0.0122,0.0525,0.0101,0.0084,0.0122,0.0175,-0.0006,0.0034,0.0118,0.0106
1997-03,0.0078,-0.0021,-0.0012,-0.0120,0.0016,-0.0023,0.0109,-0.0119,-0.0084,0.0060,0.0010,-0.0077
1997-04,0.0086,-0.0170,0.0030,0.0119,0.0119,-0.0005,0.0130,0.0172,0.0084,-0.0001,0.0122,0.0009
1997-05,0.0156,-0.0015,0.0233,0.0315,0.0189,0.0346,0.0118,0.0108,0.0394,0.0197,0.0173,0.0275


In [4]:
pd.concat([hfi.mean(),hfi.median(),hfi.mean()>hfi.median()], axis = "columns")

,0,1,2
Convertible Arbitrage,0.005781,0.00650,False
CTA Global,0.004284,0.00190,True
Distressed Securities,0.006837,0.00880,False
Emerging Markets,0.006740,0.00985,False
Equity Market Neutral,0.004330,0.00465,False
Event Driven,0.006660,0.00875,False
Fixed Income Arbitrage,0.004404,0.00550,False
Global Macro,0.005539,0.00465,True
Long/Short Equity,0.006700,0.00815,False
Merger Arbitrage,0.005569,0.00590,False


Formula for skewness

$$ S(R) = \frac{E[ (R-E(R))^3 ]}{\sigma_R^3} $$

In [5]:
erk.skewness(hfi).sort_values()

Fixed Income Arbitrage   -3.784087
Convertible Arbitrage    -2.598859
Relative Value           -2.079472
Equity Market Neutral    -1.918140
Event Driven             -1.881176
Distressed Securities    -1.732838
Merger Arbitrage         -1.620384
Emerging Markets         -1.223409
Funds Of Funds           -0.597301
Long/Short Equity        -0.468476
CTA Global                0.167094
Global Macro              0.884331
dtype: float64

In [6]:
!pip install scipy


[notice] A new release of pip is available: 23.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
# now we'll use scipy to see if we get the same results
import scipy.stats
scipy.stats.skew(hfi)

array([-2.5988594 ,  0.16709409, -1.73283848, -1.22340855, -1.91814041,
       -1.88117628, -3.78408679,  0.88433107, -0.46847577, -1.62038409,
       -2.07947152, -0.59730067])

In [8]:
erk.skewness(hfi)

Convertible Arbitrage    -2.598859
CTA Global                0.167094
Distressed Securities    -1.732838
Emerging Markets         -1.223409
Equity Market Neutral    -1.918140
Event Driven             -1.881176
Fixed Income Arbitrage   -3.784087
Global Macro              0.884331
Long/Short Equity        -0.468476
Merger Arbitrage         -1.620384
Relative Value           -2.079472
Funds Of Funds           -0.597301
dtype: float64

In [11]:
hfi.shape

(294, 12)

In [12]:
# as a further check of our code, we will generate a bunch of random normally distributed returns and try our code on those to see if we get a result of zero
import numpy as np
normal_rets = np.random.normal(0,0.15,size=(294,1)) #return eries of 294 rows, 1 column (to match the number of rows in the hfi dataframe)


In [13]:
erk.skewness(normal_rets) #check if the result is close to zero

0.05447287279152037

## Kurtosis

$$ K(R) = \frac{E[ (R-E(R))^4 ]}{\sigma_R^4} $$

In [15]:
erk.kurtosis(normal_rets) # checking is kurtosis is close to 3 (which we would expect to see for a normal distribution)

2.8824784533354557

In [16]:
# kurtosis of our hfi data:
erk.kurtosis(hfi)

Convertible Arbitrage     21.661630
CTA Global                 2.999865
Distressed Securities     10.833276
Emerging Markets           9.044334
Equity Market Neutral     15.469866
Event Driven              13.308981
Fixed Income Arbitrage    28.476464
Global Macro               5.474004
Long/Short Equity          4.916137
Merger Arbitrage          15.806243
Relative Value            13.196580
Funds Of Funds             7.420199
dtype: float64

In [17]:
# now we use scipy.stats to calculate the kurtosis to see if we get the same results:
scipy.stats.kurtosis(normal_rets) # scipy.stats.kurtosis gives the EXCESS kurtosis, ie the difference from 3

array([-0.11752155])

In [18]:
# now we will test for kurtosis more scientifically by using the Jarque Bera test from scipy.stats
scipy.stats.jarque_bera(normal_rets)

SignificanceResult(statistic=0.3145859952856258, pvalue=0.8544536692267201)

The above result shows a p-value of around 0.85, which is quite close to 1 -- indicating that there is a high level of confidence that the returns in normal_rets are normally distributed

In [19]:
scipy.stats.jarque_bera(hfi) # when we run this, we will get an incomprehensible result, since the Jarque Bera test will just treat all the data in all the columns as one giant set of returns together

SignificanceResult(statistic=18819.40862140897, pvalue=0.0)

Below we will run the is_normal function that we have defined in the erk module

In [20]:
erk.is_normal(normal_rets)

True

In [21]:
hfi.aggregate(erk.is_normal) # applying the is_normal function on each column of the hfi dataframe

Convertible Arbitrage     False
CTA Global                 True
Distressed Securities     False
Emerging Markets          False
Equity Market Neutral     False
Event Driven              False
Fixed Income Arbitrage    False
Global Macro              False
Long/Short Equity         False
Merger Arbitrage          False
Relative Value            False
Funds Of Funds            False
dtype: bool

In [22]:
ffme = erk.get_ffme_returns()
erk.skewness(ffme)

c:\Users\shantala.mukherjee\OneDrive - Invartis Consulting\Python\edhec_risk_kit.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  me_m = pd.read_csv("Portfolios_Formed_on_ME_monthly_EW.csv",


SmallCap    4.306950
LargeCap    0.205293
dtype: float64

In [23]:
erk.kurtosis(ffme)

SmallCap    46.133119
LargeCap    10.320801
dtype: float64

In [24]:
ffme.aggregate(erk.is_normal)

SmallCap    False
LargeCap    False
dtype: bool